In [ ]:
import pandas as pd
import random
# Assuming FlagEmbedding and Pinecone are custom or external libraries you have access to.
from FlagEmbedding import FlagModel
from pinecone import Pinecone

# Prepare test data
df_test = pd.read_csv('./evaluation.csv')
test = []

for index, row in df_test.iterrows():
    small_list = [row["Question"], row["PMID"]]
    test.append(small_list)
random.seed(42)
random.shuffle(test)
test = test[:100]  # Select the first 100 rows for testing

# Initialize the model
model = FlagModel('bge_large_fin', 
                  query_instruction_for_retrieval="Represent this sentence for searching relevant passages:",
                  use_fp16=True)

# Initialize Pinecone index
pc = Pinecone(api_key="621f7574-8c97-4f46-8c5e-186dd099d33b")
index = pc.Index("bge-fin")

# Function to get the relevant PMID for each question
def get_relevant_pmid(question):
    query_vector = model.encode_queries([question])[0].tolist()
    response = index.query(
        vector=query_vector,
        top_k=1,  # Request top 1 result
        include_metadata=True  
    )

    pmid = response['matches'][0]['metadata']['pmid'] if response['matches'] else None
    return pmid

# Calculate recall rate
correct_predictions = 0
total_questions = len(test)

for question, original_pmid in test:
    relevant_pmid = get_relevant_pmid(question)
    original_pmid = str(original_pmid)  # Ensure the format is consistent
    
    # Calculate recall rate
    if relevant_pmid == original_pmid:
        correct_predictions += 1

# Calculate and print the average recall rate (accuracy)
avg_recall = correct_predictions / total_questions
print(f"Average Recall (Accuracy): {avg_recall:.2f}")


Average Recall (Accuracy): 0.75
